In [8]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

import os, glob, re

In [16]:
# Define a function to pre-process the files
def process_file(file_path):    
    def read_csv_with_fallback(file_path):
        encodings = ['utf-8', 'utf-8-sig', 'latin1', 'cp1252']
        for enc in encodings:
            try:
                df = pd.read_csv(file_path, delimiter=",", encoding=enc)
                print(f"Successfully read {file_path} with encoding: {enc}")
                return df
            except UnicodeDecodeError:
                print(f"Failed with encoding: {enc}")
                continue
        raise UnicodeDecodeError(f"Could not decode {file_path} with tried encodings.") 
    df = read_csv_with_fallback(file_path)  

    df.columns = df.columns.str.strip()

    # remove columns which have the 'Unnamed:' 
    df = df.loc[:, ~df.columns.str.startswith('Unnamed:')]
    
    # add a column to keep track of the csv file in which each entry came from
    base_name = os.path.basename(file_path)
    date_match = re.search(r'(\d{8})', base_name)
    if date_match:
        file_date = pd.to_datetime(date_match.group(1), format='%Y%m%d').date()
    else:
        file_date = None  # fallback if no date in filename
    df["file_name"] = file_date

    # drop the 'Vs Old' column, it has no meaning
    if 'Vs Old' in df.columns:
        df.drop(columns=['Vs Old'], inplace=True)

    # certain columns have \n and \r, so they are removed
    df = df.replace({r"\r|\n": " "}, regex=True)
    df.columns = df.columns.str.replace(r'[\r\n]', ' ', regex=True).str.replace(r'\s+', ' ', regex=True).str.strip()

    # pre-processing on the columns
    # extract information from the 'start' and 'finish' columns and separate into separate columns
    df['Start'] = df['Start'].replace({r'\r|\n': ' '}, regex=True)
    df['Finish'] = df['Finish'].replace({r'\r|\n': ' '}, regex=True)
    df['Start'] = pd.to_datetime(df['Start'], dayfirst=True, format='%d/%m/%Y %H:%M %A', errors='coerce')
    df['Finish'] = pd.to_datetime(df['Finish'], dayfirst=True, format='%d/%m/%Y %H:%M %A', errors='coerce')

    # extract information from 'recall'
    pattern = r"Day:\s*(\d+)\s*hr?s?(?:-Night:\s*(\d+)\s*hr?s?)?"
    df[["Recall_Day_Hours", "Recall_Night_Hours"]] = df["Recall"].str.extract(pattern)
    df["Recall_Day_Hours"] = pd.to_numeric(df["Recall_Day_Hours"], errors="coerce")
    df['Recall_Day_Hours'] = df['Recall_Day_Hours'].fillna(0)
    df["Recall_Night_Hours"] = pd.to_numeric(df["Recall_Night_Hours"], errors="coerce")
    df['Recall_Night_Hours'] = df['Recall_Night_Hours'].fillna(0)






    # extract information for 'Impact region, reason and duration'
    # this column only appears as of 28/03/2022
    # Default: start with Impact_Region = Region

    # Define a list of known region codes
    valid_regions = ['NSW', 'QLD', 'VIC', 'SA', 'TAS']
    region_pattern = r'\b(?:' + '|'.join(valid_regions) + r')\b\s*'

    source_col = None

    if "Impact Region, Reason and Duration" in df.columns:
        source_col = "Impact Region, Reason and Duration"
    elif "Reason and Duration" in df.columns:
        source_col = "Reason and Duration"

    if source_col:
        # Extract Reason and Duration
        pattern = r"([a-zA-Z\s]+?)\s+([\d.]+)\s*(Days?|Hours?|Minutes?)"
        extracted = df[source_col].str.extract(pattern)
        extracted.columns = ['Reason', 'Value', 'Unit']

        # Clean up Reason, Value, and Unit
        extracted["Reason"] = extracted["Reason"].str.strip()
        
        extracted["Value"] = pd.to_numeric(extracted["Value"], errors='coerce')
        df["Reason"] = extracted["Reason"]
        df['Reason'] = df['Reason'].str.replace(region_pattern, '', regex=True).str.strip()

        df["Value"] = extracted["Value"]
        df["Unit"] = extracted["Unit"]

        # Calculate Duration in Hours (based on 'Duration' column if you prefer that)
        if "Duration" in df.columns:
            duration_pattern = r"([\d\.]+)\s*(Days?|Hours?|Minutes?)"
            dur_extracted = df["Duration"].str.extract(duration_pattern)
            dur_extracted.columns = ["Value", "Unit"]
            dur_extracted["Value"] = pd.to_numeric(dur_extracted["Value"], errors="coerce")
            df["Duration_Hours"] = dur_extracted.apply(
                lambda row: row["Value"] * 24 if pd.notnull(row["Unit"]) and "Day" in row["Unit"]
                else row["Value"] if pd.notnull(row["Unit"]) and "Hour" in row["Unit"]
                else row["Value"] / 60 if pd.notnull(row["Unit"]) and "Minute" in row["Unit"]
                else None, axis=1)
        else:
            df["Duration_Hours"] = df.apply(
                lambda row: row["Value"] * 24 if pd.notnull(row["Unit"]) and "Day" in row["Unit"]
                else row["Value"] if pd.notnull(row["Unit"]) and "Hour" in row["Unit"]
                else row["Value"] / 60 if pd.notnull(row["Unit"]) and "Minute" in row["Unit"]
                else None, axis=1)

        df["Duration_Hours"] = df["Duration_Hours"].round(2)

        # Extract Impact_Region from known state codes
        def extract_regions(text):
            return " / ".join(re.findall(r"\b(?:NSW|QLD|VIC|SA|TAS)\b", str(text)))

        df["Impact_Region"] = df[source_col].apply(extract_regions)
        df.loc[df["Impact_Region"] == "", "Impact_Region"] = df["Region"]








    # split information from 'status'
    df[["Status_Description", "Status_Code"]] = df["Status"].str.split(" - ", expand=True)
    # Split information from 'status and market notice'
    df[['Status_Description_Market', 'Market_Notice_Code']] = df['Status and Market Notice'].str.split(" - ", n=1, expand=True)
    
    # Clean the column by removing "MN: " and "Issued on: " parts and keeping only the codes
    df["Market_Notice_Code"] = df["Market_Notice_Code"].str.replace(r"^(STLTP|MTLTP|RESUBMIT|PTP|PDLTP|PTR|UTP)\s*(?:MN: \d+ - Issued on:.*)?", r"\1", regex=True)


    # List of boolean columns
    bool_cols = ['Project Work?', 'Unplanned?', 'Generator Aware?', 'DNSP Aware?', 'Inter-Regional']
    # Replace 'T' with 1 and NaN with 0
    df[bool_cols] = df[bool_cols].apply(lambda col: col.map(lambda x: 1 if x == 'T' else 0))

    # drop non-needed columns now only if they're in the list
    cols_to_drop = ['Status', 'Status and Market Notice', 'Duration', 'Value', 'Unit',
                    'Reason and Duration', 'Recall']
    df.drop(columns=[col for col in cols_to_drop if col in df.columns], inplace=True)

    # re-order columns 
    # cols = list(df.columns) 
    new_col_order = ['Region', 'NSP', 
                    'Start', 'Finish', # Start and Finish information
                    'Network Asset', # identifying information
                    'Recall_Day_Hours', 'Recall_Night_Hours', # recall information
                    'Project Work?', 'Unplanned?', 'DNSP Aware?', 'Generator Aware?', 'Inter-Regional', # boolean terms
                    'Status_Description', 'Status_Code', 'Status_Description_Market', 'Market_Notice_Code', # status information
                    'Impact_Region', 'Reason', 'Duration_Hours', # reason and duration information
                    'Impact', 'file_name']
    new_col_order = list(dict.fromkeys(new_col_order))
    df = df[new_col_order]

    # remove rows where 'Region' is empty
    df = df[~df['Region'].str.startswith("The following fields of data are submitted by NSPs to AEMO.", na=False)]
    df = df.dropna(subset=['Region'])

    return df


In [17]:
# List of CSV files to process
csv_files = glob.glob("aemo_data/*.csv")

# List to store processed dataframes
processed_dfs = []

# Loop through each file, apply the processing function and collect the results
for file in csv_files:
    processed_df = process_file(file)
    processed_dfs.append(processed_df)

# there is an error with 20230109 and 20230314

Failed with encoding: utf-8
Failed with encoding: utf-8-sig
Successfully read aemo_data\High_Impact_Outages_20210830.csv with encoding: latin1
Failed with encoding: utf-8
Failed with encoding: utf-8-sig
Successfully read aemo_data\High_Impact_Outages_20210906.csv with encoding: latin1
Successfully read aemo_data\High_Impact_Outages_20210913.csv with encoding: utf-8
Successfully read aemo_data\High_Impact_Outages_20210920.csv with encoding: utf-8
Failed with encoding: utf-8
Failed with encoding: utf-8-sig
Successfully read aemo_data\High_Impact_Outages_20210927.csv with encoding: latin1
Failed with encoding: utf-8
Failed with encoding: utf-8-sig
Successfully read aemo_data\High_Impact_Outages_20211005.csv with encoding: latin1
Failed with encoding: utf-8
Failed with encoding: utf-8-sig
Successfully read aemo_data\High_Impact_Outages_20211011.csv with encoding: latin1
Successfully read aemo_data\High_Impact_Outages_20211018.csv with encoding: utf-8
Successfully read aemo_data\High_Impact

In [18]:
# concatenate all dataframes and export to csv
full_df = pd.concat(processed_dfs, ignore_index=True)
full_df.info()
full_df.to_csv("processed_high_impact_outages.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18714 entries, 0 to 18713
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Region                     18714 non-null  object        
 1   NSP                        18714 non-null  object        
 2   Start                      18504 non-null  datetime64[ns]
 3   Finish                     18504 non-null  datetime64[ns]
 4   Network Asset              18714 non-null  object        
 5   Recall_Day_Hours           18714 non-null  float64       
 6   Recall_Night_Hours         18714 non-null  float64       
 7   Project Work?              18714 non-null  int64         
 8   Unplanned?                 18714 non-null  int64         
 9   DNSP Aware?                18714 non-null  int64         
 10  Generator Aware?           18714 non-null  int64         
 11  Inter-Regional             18714 non-null  int64         
 12  Stat

In [19]:
for col in full_df.columns:
    print(full_df[col].value_counts())
    print('')

Region
VIC         10099
QLD          3932
SA           2541
NSW          1122
TAS          1019
VIC/ TAS        1
Name: count, dtype: int64

NSP
AusNet         9991
Powerlink      3932
ElectraNet     2541
Transgrid      1122
TasNetworks    1019
TOA             109
Name: count, dtype: int64

Start
2025-04-23 08:00:00    229
2025-09-02 08:00:00    111
2023-12-31 06:00:00     96
2024-06-04 06:00:00     93
2024-10-09 07:00:00     86
                      ... 
2022-11-12 04:01:00      1
2022-11-13 05:00:00      1
2022-11-12 05:00:00      1
2022-11-11 05:00:00      1
2025-06-17 08:00:00      1
Name: count, Length: 1597, dtype: int64

Finish
2025-09-15 17:00:00    170
2025-05-09 17:00:00    112
2023-12-31 16:00:00     96
2024-06-04 18:00:00     93
2024-10-21 17:00:00     88
                      ... 
2022-07-11 10:15:00      1
2022-07-12 11:30:00      1
2022-06-29 17:00:00      1
2024-07-21 17:30:00      1
2025-06-27 17:00:00      1
Name: count, Length: 1614, dtype: int64

Network Asset
Moor

In [22]:
# Inspect rows where 'Region' is null
null_region_rows = full_df[full_df['Duration_Hours'].isnull()]

# Display the rows with null values in the 'Region' column
print(null_region_rows)

      Region          NSP               Start              Finish  \
1005     TAS  TasNetworks 2022-01-11 07:00:00 2022-01-11 15:00:00   
6824     QLD    Powerlink 2023-03-29 07:00:00 2023-03-29 15:00:00   
9039     QLD    Powerlink 2023-09-04 07:00:00 2023-10-11 17:00:00   
10109     SA   ElectraNet 2023-11-01 17:30:00 2023-11-02 05:30:00   
10110     SA   ElectraNet 2023-11-02 17:30:00 2023-11-03 05:30:00   
10246     SA   ElectraNet 2023-11-01 17:30:00 2023-11-02 05:30:00   
10249     SA   ElectraNet 2023-11-02 17:30:00 2023-11-03 05:30:00   
10376     SA   ElectraNet 2023-11-01 17:30:00 2023-11-02 05:30:00   
10379     SA   ElectraNet 2023-11-02 17:30:00 2023-11-03 05:30:00   

                                  Network Asset  Recall_Day_Hours  \
1005        Gordon - Chapel St No.2 220 kV Line               2.0   
6824        Ross No.3 288/138/19 kV Transformer               8.0   
9039         Bouldercmb - Nebo (821) 275kV Line               0.0   
10109  Tailem Bend - Tungkillo No